# GPT-OSS-120B Tools Demo: Clean Implementation

This notebook demonstrates how to use Fireworks' GPT-OSS-120B model with code interpreter functionality.

## Features
- ✅ **Persistent Code Execution**: Variables persist across tool calls
- ✅ **Proper Argument Parsing**: Handles both JSON and plain text formats
- ✅ **Complete Tool Flow**: Multi-iteration conversations with tools
- ✅ **Mathematical Calculations**: Complex matrix operations and eigenvalue calculations

## Prerequisites
- Fireworks API key set in environment variable `FIREWORKS_API_KEY`
- Python packages: `requests`, `sympy`, `numpy`

## 📋 Setup Instructions
**Run cells in order:**
1. Install packages
2. Import libraries and check API key
3. Define helper functions
4. Run demo


In [1]:
# Install required packages
%pip install -q requests sympy numpy matplotlib



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
# Import libraries and check API key
import requests
import os
import json
import math
import numpy as np
from typing import Dict, List, Any

# Check if API key is available
api_key = os.getenv('FIREWORKS_API_KEY')
if not api_key:
    raise ValueError("Please set FIREWORKS_API_KEY environment variable")

print("✅ Fireworks API key loaded successfully")
print("✅ All libraries imported")


✅ Fireworks API key loaded successfully
✅ All libraries imported


In [3]:
# Global persistent namespace for code execution
_persistent_namespace = None

def get_persistent_namespace():
    """Get or create the persistent namespace for code execution"""
    global _persistent_namespace
    if _persistent_namespace is None:
        import sympy as sp
        import matplotlib.pyplot as plt
        try:
            import pandas as pd
        except ImportError:
            pd = None
        
        _persistent_namespace = {
            '__builtins__': __builtins__,
            'math': math,
            'np': np,
            'numpy': np,
            'sp': sp,
            'sympy': sp,
            'plt': plt,
        }
        if pd:
            _persistent_namespace['pd'] = pd
            _persistent_namespace['pandas'] = pd
    return _persistent_namespace

def reset_persistent_namespace():
    """Reset the persistent namespace (call this when starting a new conversation)"""
    global _persistent_namespace
    _persistent_namespace = None

def execute_python_code(code: str) -> str:
    """
    Execute Python code with persistent namespace across calls
    """
    try:
        print(f"🐍 Executing Python code:")
        print(f"--- CODE START ---")
        print(code)
        print(f"--- CODE END ---")
        
        # Get the persistent namespace
        namespace = get_persistent_namespace()
        
        # Split code into lines for analysis
        lines = [line.strip() for line in code.split('\n') if line.strip()]
        if not lines:
            return "No code to execute"
        
        last_line = lines[-1]
        
        # Check if the last line is an expression (not an assignment or statement)
        is_expression = (last_line and 
                        not any(keyword in last_line for keyword in ['import', 'def', 'class', 'if', 'for', 'while', 'print', 'plt.']) and
                        '=' not in last_line and
                        not last_line.endswith(':'))
        
        print(f"🔍 Last line: '{last_line}' | Is expression: {is_expression}")
        
        if is_expression:
            # Execute all lines except the last one
            if len(lines) > 1:
                setup_code = '\n'.join(lines[:-1])
                print(f"📝 Executing setup code...")
                exec(setup_code, namespace)
            
            # Evaluate the last line as an expression
            try:
                print(f"🎯 Evaluating expression: {last_line}")
                result = eval(last_line, namespace)
                result_str = str(result)
                print(f"✅ Expression result: {result_str[:200]}{'...' if len(result_str) > 200 else ''}")
                return result_str
            except Exception as eval_error:
                print(f"⚠️ Expression evaluation failed: {eval_error}")
                # If evaluation fails, execute the whole thing and look for variables
                exec(code, namespace)
                return f"Code executed. Expression evaluation failed: {str(eval_error)}"
        else:
            # Execute the entire code block
            print(f"📝 Executing entire code block...")
            exec(code, namespace)
            
            # Look for recently defined variables in reverse order of preference
            recent_vars = ['result', 'output', '_', 'ans', 'answer']
            for var_name in recent_vars:
                if var_name in namespace:
                    try:
                        val = namespace[var_name]
                        # Skip functions and modules
                        if not callable(val) and not hasattr(val, '__module__'):
                            result_str = str(val)
                            print(f"✅ Found variable '{var_name}': {result_str[:200]}{'...' if len(result_str) > 200 else ''}")
                            return result_str
                    except:
                        continue
            
            # Look for any variables that might have been created/modified
            # Check for common mathematical result patterns
            math_vars = ['C', 'A', 'B', 'eigenvals', 'matrix', 'M', 'result_matrix']
            for var_name in math_vars:
                if var_name in namespace:
                    try:
                        val = namespace[var_name]
                        if not callable(val) and str(type(val)).startswith('<class'):
                            result_str = str(val)
                            print(f"✅ Found math variable '{var_name}': {result_str[:200]}{'...' if len(result_str) > 200 else ''}")
                            return result_str
                    except:
                        continue
            
            print("ℹ️ No explicit result to display")
            return "Code executed successfully"
        
    except Exception as e:
        error_msg = f"Error executing code: {str(e)}"
        print(f"❌ {error_msg}")
        return error_msg

print("✅ Code execution functions defined")


✅ Code execution functions defined


In [4]:
def call_gpt_oss_120b(messages: List[Dict], tools: List[Dict] = None, max_tokens: int = 4096) -> Dict:
    """
    Call the GPT-OSS-120B model via Fireworks API
    """
    url = "https://api.fireworks.ai/inference/v1/chat/completions"
    
    payload = {
        "model": "accounts/fireworks/models/gpt-oss-120b",
        "messages": messages,
        "max_tokens": max_tokens,
        "stream": False,
    }
    
    if tools:
        payload["tools"] = tools
    
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json",
    }
    
    print(f"🌐 Calling Fireworks API...")
    response = requests.post(url, json=payload, headers=headers)
    response.raise_for_status()
    
    return response.json()

def handle_tool_calls(messages: List[Dict], tools: List[Dict], max_iterations: int = 5) -> List[Dict]:
    """
    Handle a complete conversation with tool calls
    """
    iteration = 0
    
    while iteration < max_iterations:
        iteration += 1
        print(f"\n{'='*20} Iteration {iteration} {'='*20}")
        
        # Call the model
        response = call_gpt_oss_120b(messages, tools=tools)
        assistant_message = response['choices'][0]['message']
        
        print(f"🤖 Assistant Response:")
        if 'reasoning_content' in assistant_message:
            print(f"💭 Reasoning: {assistant_message['reasoning_content'][:300]}...")
        
        messages.append(assistant_message)
        
        # Check if there are tool calls to handle
        if 'tool_calls' in assistant_message and assistant_message['tool_calls']:
            print(f"🛠️ Processing {len(assistant_message['tool_calls'])} tool call(s)")
            
            for tool_call in assistant_message['tool_calls']:
                if tool_call['type'] == 'function':
                    function_name = tool_call['function']['name']
                    arguments = tool_call['function']['arguments']
                    
                    print(f"📞 Calling function: {function_name}")
                    
                    if function_name == 'python':
                        # Handle different argument formats
                        if isinstance(arguments, str):
                            # Try to parse as JSON first
                            try:
                                args_dict = json.loads(arguments)
                                if 'code' in args_dict:
                                    python_code = args_dict['code']
                                else:
                                    python_code = arguments  # Use raw string
                            except json.JSONDecodeError:
                                python_code = arguments  # Use raw string
                        else:
                            python_code = str(arguments)
                        
                        # Fix newline escaping issues
                        python_code = python_code.replace('\\n', '\n')
                        
                        # Execute the Python code
                        result = execute_python_code(python_code)
                        
                    else:
                        result = f"Unknown function: {function_name}"
                    
                    # Add tool response
                    tool_response = {
                        "role": "tool",
                        "content": result,
                        "tool_call_id": tool_call['id']
                    }
                    print(f"📤 Tool response: {result[:200]}{'...' if len(result) > 200 else ''}")
                    messages.append(tool_response)
            
            # Continue to next iteration
            continue
            
        else:
            # No more tool calls - conversation is complete
            print("✅ Conversation complete - no more tool calls needed")
            if 'content' in assistant_message and assistant_message['content']:
                print(f"\n💬 Final response:\n{assistant_message['content']}")
            break
    
    if iteration >= max_iterations:
        print(f"\n⚠️ Reached maximum iterations ({max_iterations})")
    
    return messages

print("✅ API and conversation handling functions defined")


✅ API and conversation handling functions defined


In [5]:
# Quick verification that all functions work
print("🔍 Testing helper functions...")

# Test namespace functions
reset_persistent_namespace()
namespace = get_persistent_namespace()
print(f"✅ Persistent namespace: {list(namespace.keys())[:5]}...")

# Test code execution
test_result = execute_python_code("x = 5\ny = x * 2\ny")
print(f"✅ Code execution test result: {test_result}")

print("\n🎉 All functions are ready!")


🔍 Testing helper functions...


✅ Persistent namespace: ['__builtins__', 'math', 'np', 'numpy', 'sp']...
🐍 Executing Python code:
--- CODE START ---
x = 5
y = x * 2
y
--- CODE END ---
🔍 Last line: 'y' | Is expression: True
📝 Executing setup code...
🎯 Evaluating expression: y
✅ Expression result: 10
✅ Code execution test result: 10

🎉 All functions are ready!


In [6]:
# Define the code interpreter tool
code_interpreter_tool = {
    "type": "function",
    "function": {
        "name": "python",
        "description": "Execute Python code and return the result",
        "parameters": {
            "type": "object",
            "properties": {
                "code": {
                    "type": "string",
                    "description": "Python code to execute"
                }
            },
            "required": ["code"]
        }
    }
}

# Mathematical problem for the model to solve
math_query = """
Calculate the following complex mathematical expression and show your work:

1. Compute the matrix multiplication of two 3x3 matrices:
   A = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]
   B = [[9, 8, 7], [6, 5, 4], [3, 2, 1]]
2. Find the eigenvalues of the resulting matrix

Please show all intermediate steps and verify your calculations.
"""

# Reset namespace for fresh start
reset_persistent_namespace()

# Initial message
messages = [{"role": "user", "content": math_query}]

print("🚀 Starting GPT-OSS-120B mathematical calculation demo...")
print("🎯 Problem: Matrix multiplication and eigenvalue calculation")
print("🔧 Using persistent namespace to maintain variables across tool calls")

# Handle the complete conversation
final_messages = handle_tool_calls(messages, [code_interpreter_tool], max_iterations=10)

print(f"\n🏁 Demo completed! Total messages: {len(final_messages)}")


🚀 Starting GPT-OSS-120B mathematical calculation demo...
🎯 Problem: Matrix multiplication and eigenvalue calculation
🔧 Using persistent namespace to maintain variables across tool calls

==================== Iteration 1 ====================
🌐 Calling Fireworks API...
🤖 Assistant Response:
💭 Reasoning: We need to compute A*B, then eigenvalues of resulting matrix. Show steps. Let's compute using python to verify....
🛠️ Processing 1 tool call(s)
📞 Calling function: python
🐍 Executing Python code:
--- CODE START ---
import sympy as sp
A = sp.Matrix([[1,2,3],[4,5,6],[7,8,9]])
B = sp.Matrix([[9,8,7],[6,5,4],[3,2,1]])
C = A*B
C

--- CODE END ---
🔍 Last line: 'C' | Is expression: True
📝 Executing setup code...
🎯 Evaluating expression: C
✅ Expression result: Matrix([[30, 24, 18], [84, 69, 54], [138, 114, 90]])
📤 Tool response: Matrix([[30, 24, 18], [84, 69, 54], [138, 114, 90]])

==================== Iteration 2 ====================
🌐 Calling Fireworks API...
🤖 Assistant Response:
💭 Reasoning: 

## Additional Examples

Here are some other mathematical problems you can try:
